# Baseline: ecapa

In [5]:
from speechbrain.nnet.losses import LogSoftmaxWrapper, AdditiveAngularMargin
from speechbrain.pretrained import EncoderClassifier
from data import create_dataloader
import torch
import numpy as np
from sklearn.metrics import classification_report, RocCurveDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
import os
import matplotlib.pyplot as plt

## Configuration

In [6]:
model_name='spkrec-ecapa-voxceleb'
dir='recording'
root=os.path.join('dataset',model_name,dir)

## Data preparation

In [7]:
def compute_features(model, loader, criterion):
    features=[]
    labels=[]
    lab2ind=model.hparams.label_encoder.lab2ind
    for i, data in enumerate(loader):

        x, label, y = data
        x = x.cuda()
        with torch.inference_mode():
            pred, score, _, _ = model.classify_batch(x)
        features.append([score])
        labels.append(label)
    return torch.tensor(features).numpy(), torch.tensor(labels).numpy()


In [8]:
if os.path.exists(root):
    train=np.load(os.path.join(root,'train.npy'))
    val=np.load(os.path.join(root,'val.npy'))
    test=np.load(os.path.join(root,'test.npy'))
else:
    os.makedirs(root)
    criterion=LogSoftmaxWrapper(AdditiveAngularMargin(margin=0.2, scale=30))
    model=EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", run_opts={"device": "cuda"})
    model.eval()
    train_loader, val_loader, test_loader = create_dataloader(
        root='split/recording',
        data_root='/home/baole/cmu/2023spring/11785/data/vox1/vox1_all/wav/',
        batch_size=1
    )

    train_x, train_y=compute_features(model, train_loader, criterion)
    val_x, val_y=compute_features(model, val_loader, criterion)
    test_x, test_y=compute_features(model, test_loader, criterion)

    train=np.column_stack((train_x,train_y))
    np.save(os.path.join(root,'train.npy'),train)
    val=np.column_stack((val_x,val_y))
    np.save(os.path.join(root,'val.npy'),val)
    test=np.column_stack((test_x,test_y))
    np.save(os.path.join(root,'test.npy'),test)

KeyError: 'label'

## Visualization

In [ ]:
def visualize(dataset):
    plt.scatter(dataset[:,0],dataset[:,1])
    plt.xlabel('confidence')
    plt.ylabel('label (pos/neg)')
    plt.show()

In [ ]:
visualize(train)

In [ ]:
visualize(val)

In [ ]:
visualize(test)

## Evaluation

In [ ]:
def classify(classifier, train, test):
    classifier.fit(train[:,:-1], train[:,-1])
    pred=classifier.predict(test[:,:-1])
    print(classification_report(test[:,-1],pred))
    RocCurveDisplay.from_estimator(classifier,test[:,:-1],test[:,-1])
    plt.show()

In [ ]:
classify(LogisticRegression(),train,test)

In [ ]:
classify(SVC(),train,test)

In [ ]:
classify(RandomForestClassifier(),train,test)

In [ ]:
classify(AdaBoostClassifier(),train,test)

In [ ]:
classify(GradientBoostingClassifier(),train,test)

In [ ]:
classify(MLPClassifier(),train,test)